In [28]:
from catboost import CatBoostRegressor
import catboost
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias
import hyperopt
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

df_train = pd.read_csv('./data/train_filtrado.csv')
# Para usarse con el submit a Kaggle
df_eval = pd.read_csv('./data/test.csv')

df_train, df_eval = features.features_de_csvs(df_train, df_eval)

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_eval_idf = pd.read_csv('./data/test_idf.csv')

df_train = pd.merge(df_train, df_train_idf, on= 'id', how= 'left')
df_eval = pd.merge(df_eval, df_eval_idf, on= 'id', how= 'left')

df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.4)

df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)
df_train = features.llenar_nulls(df_train, hgb_mean=True)

In [29]:
df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

df_train_f = f_distancias.feature_distancias(df_train_f)
df_test_f = f_distancias.feature_distancias(df_test_f, df_train_f)

In [30]:
df_train_f = features.features_intervalos(df_train_f)
df_test_f = features.features_intervalos(df_test_f)

In [6]:
# # Paso a integer las features

# def feature_convertion(df,features, multiplicar):
#     df = df.copy()
    
#     for feature in features:
#         if multiplicar:
#             df[feature] = df[feature].map(lambda x: x*1000)
#         df[feature] = df[feature].astype('int64')
        
#     return df


# features_int = ['ditancia_centro_bins_unif','ditancia_centro_bins_perc','distancia_centro_mexico_bins_unif',
#             'distancia_centro_mexico_bins_perc','distancia_ciudad_cara_bins_unif',
#             'distancia_ciudad_cara_bins_perc','promedio_id_zona_bins_unif', 'promedio_id_zona_bins_perc',
#             'promedio_precio_hbg_tipo_propiedad_provincia_bins_unif',
#             'promedio_precio_hbg_tipo_propiedad_provincia_bins_perc', 
#             'promedio_precio_tipo_propiedad_ciudad_bins_unif','promedio_precio_tipo_propiedad_ciudad_bins_perc',
#             'antiguedad_bins_unif', 'antiguedad_bins_perc', 'promedio_precio_hbg_tipo_propiedad_provincia_bins_perc',
#             'promedio_precio_hbg_tipo_propiedad_provincia_bins_unif', 'tam_ambientes_bins_perc',
#             'promedio_precio_hbg_tipo_propiedad_provincia_bins_unif', 'habitaciones', 'banos',
#             'metroscubiertos', 'metrostotales', 'count_id_zona', 'antiguedad','garages', 'gimnasio',
#             'usosmultiples', 'piscina', 'gimnasio']

# df_train_f = feature_convertion(df_train_f, features_int, False)

In [7]:
# features_mult = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
#        'metrostotales', 'lat', 'lng', 'fecha', 'piscina', 'escuelascercanas',
#        'centroscomercialescercanos', 'clustering_label', 'idf_titulo',
#        'idf_descripcion', 'peso_titulo', 'peso_descripcion',
#        'porcentaje_metros', 'diferencia_metros', 'metros_totales_normalizados',
#        'metros_cubiertos_normalizados', 'escomercial',
#        'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
#        'prop_frecuente', 'top_provincia', 'es_ciudad_centrica',
#        'promedio_metros_totales_provincia',
#        'promedio_metros_cubiertos_provincia', 'mes', 'dia', 'trimestre',
#        'dias_desde_datos', 'meses_desde_datos', 'antiguedad_bins_perc',
#        'cantidad_inquilinos', 'tam_ambientes', 'promedio_precio_provincia',
#        'promedio_precio_ciudad', 'varianza_precio_ciudad', 'count_ciudad',
#        'promedio_id_zona', 'promedio_id_zona_gen', 'varianza_id_zona',
#        'count_id_zona', 'promedio_precio_tipo_propiedad',
#        'promedio_precio_tipo_propiedad_ciudad',
#        'promedio_precio_tipo_propiedad_ciudad_gen', 'count_tipo_propiedad',
#        'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes',
#        'promedio_precio_habitaciones',
#        'promedio_precio_habitaciones_banos_garages',
#        'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
#        'lat_norm', 'lng_norm', 'promedio_precio_booleanos', 'puntaje',
#        'distancia_ciudad_centrica', 'distancia_centro_mexico',
#        'distancia_ciudad_cara']


# df_train_f = feature_convertion(df_train_f, features_mult, True)

# df_train_f['lat'] = df_train_f['lat'].map(lambda x: x*1000000)
# df_train_f['lng'] = df_train_f['lng'].map(lambda x: x*1000000)
# df_train_f['lat'] = df_train_f['lng'].astype('int64')
# df_train_f['lng'] = df_train_f['lng'].astype('int64')

## Busqueda de Hiperparametros

### RandomizedSearchCV

In [17]:
from sklearn.model_selection import RandomizedSearchCV


features = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'lat', 'lng', 'fecha', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'clustering_label', 'idf_titulo',
       'idf_descripcion', 'peso_titulo', 'peso_descripcion',
       'porcentaje_metros', 'diferencia_metros', 'metros_totales_normalizados',
       'metros_cubiertos_normalizados', 'escomercial',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
       'prop_frecuente', 'top_provincia', 'es_ciudad_centrica',
       'promedio_metros_totales_provincia',
       'promedio_metros_cubiertos_provincia', 'mes', 'dia', 'trimestre',
       'dias_desde_datos', 'meses_desde_datos', 'antiguedad_bins_perc',
       'cantidad_inquilinos', 'tam_ambientes', 'promedio_precio_provincia',
       'promedio_precio_ciudad', 'varianza_precio_ciudad', 'count_ciudad',
       'promedio_id_zona', 'promedio_id_zona_gen', 'varianza_id_zona',
       'count_id_zona', 'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'promedio_precio_tipo_propiedad_ciudad_gen', 'count_tipo_propiedad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes',
       'promedio_precio_habitaciones',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'lat_norm', 'lng_norm', 'promedio_precio_booleanos', 'puntaje',
       'distancia_ciudad_centrica', 'distancia_centro_mexico',
       'distancia_ciudad_cara']

cat_params = {
    'learning_rate': [0.05, 0.1, 0.2],
    'depth': [6, 8, 10],
#     'bagging_temperature': [0.2],
    'od_type': ['IncToDec', 'Iter'],
#     'metric_period': [1],   # No lo encuentro, no se de donde salio
    'od_wait': [10, 30, 50, 100],
    'l2_leaf_reg': [0, 3, 5, 10, 15, 50],
    
}


x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=1)

rs_cv = RandomizedSearchCV(estimator=CatBoostRegressor(iterations=30, eval_metric='MAE', silent=True),
                           param_distributions=cat_params, 
                           cv=3, 
                           n_iter=50,
                           verbose=3, scoring='neg_mean_absolute_error')

rs_cv.fit(x_train, y_train, verbose=1)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] od_wait=30, od_type=Iter, learning_rate=0.2, l2_leaf_reg=5, depth=10 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 1359036.7860165	total: 225ms	remaining: 6.52s
1:	learn: 1170232.6961613	total: 400ms	remaining: 5.59s
2:	learn: 1025477.1023009	total: 702ms	remaining: 6.32s
3:	learn: 918104.6432642	total: 1.16s	remaining: 7.51s
4:	learn: 833105.7294456	total: 1.61s	remaining: 8.06s
5:	learn: 771017.6296518	total: 1.81s	remaining: 7.24s
6:	learn: 721633.0669587	total: 2.02s	remaining: 6.62s
7:	learn: 685901.5915150	total: 2.2s	remaining: 6.05s
8:	learn: 659169.4399477	total: 2.39s	remaining: 5.58s
9:	learn: 637039.4955827	total: 2.69s	remaining: 5.38s
10:	learn: 619628.5237321	total: 3.12s	remaining: 5.39s
11:	learn: 605965.1476844	total: 3.28s	remaining: 4.92s
12:	learn: 595305.0054124	total: 3.47s	remaining: 4.54s
13:	learn: 584953.5275728	total: 3.64s	remaining: 4.16s
14:	learn: 577650.5823283	total: 3.84s	remaining: 3.84s
15:	learn: 570550.2000684	total: 4.02s	remaining: 3.52s
16:	learn: 566142.2832398	total: 4.23s	remaining: 3.24s
17:	learn: 562218.8514279	total: 4.48s	remaining: 2.98s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.8s remaining:    0.0s


0:	learn: 1361845.3342785	total: 424ms	remaining: 12.3s
1:	learn: 1176062.4830810	total: 686ms	remaining: 9.6s
2:	learn: 1026018.2615479	total: 1.03s	remaining: 9.28s
3:	learn: 916642.1999348	total: 1.2s	remaining: 7.8s
4:	learn: 831332.3402328	total: 1.39s	remaining: 6.94s
5:	learn: 769428.7410394	total: 1.54s	remaining: 6.17s
6:	learn: 721941.7677362	total: 1.74s	remaining: 5.7s
7:	learn: 682637.8487155	total: 1.94s	remaining: 5.33s
8:	learn: 653737.2687785	total: 2.44s	remaining: 5.69s
9:	learn: 632625.7912632	total: 2.6s	remaining: 5.19s
10:	learn: 616622.0890819	total: 2.83s	remaining: 4.89s
11:	learn: 603066.3319724	total: 2.99s	remaining: 4.49s
12:	learn: 591967.0435193	total: 3.19s	remaining: 4.17s
13:	learn: 584494.1208983	total: 3.35s	remaining: 3.83s
14:	learn: 577107.0321768	total: 3.54s	remaining: 3.54s
15:	learn: 570719.5449473	total: 3.71s	remaining: 3.25s
16:	learn: 565228.6902702	total: 4.1s	remaining: 3.14s
17:	learn: 561883.0177945	total: 4.29s	remaining: 2.86s
18:	l

...

4:	learn: 855095.8561212	total: 1.57s	remaining: 7.88s
5:	learn: 790946.6527973	total: 1.75s	remaining: 7.01s
6:	learn: 742007.7986081	total: 1.94s	remaining: 6.37s
7:	learn: 705004.9354693	total: 2.14s	remaining: 5.88s
8:	learn: 676222.5588871	total: 2.67s	remaining: 6.24s
9:	learn: 653739.7346273	total: 3.3s	remaining: 6.6s
10:	learn: 636196.1234073	total: 3.78s	remaining: 6.53s
11:	learn: 621763.3986649	total: 4.07s	remaining: 6.1s
12:	learn: 610722.2463371	total: 4.23s	remaining: 5.53s
13:	learn: 602075.4134991	total: 4.62s	remaining: 5.28s
14:	learn: 594375.2736365	total: 4.95s	remaining: 4.95s
15:	learn: 588524.4789959	total: 5.39s	remaining: 4.72s
16:	learn: 582309.6701993	total: 5.56s	remaining: 4.25s
17:	learn: 577814.9347159	total: 5.96s	remaining: 3.97s
18:	learn: 573685.1494510	total: 6.14s	remaining: 3.55s
19:	learn: 570040.5668674	total: 6.32s	remaining: 3.16s
20:	learn: 567432.7938994	total: 6.49s	remaining: 2.78s
21:	learn: 564656.8985490	total: 6.86s	remaining: 2.49s
2

21:	learn: 646530.8796487	total: 2s	remaining: 729ms
22:	learn: 638895.9315897	total: 2.18s	remaining: 663ms
23:	learn: 631274.0426201	total: 2.34s	remaining: 585ms
24:	learn: 623975.1056697	total: 2.47s	remaining: 494ms
25:	learn: 618680.7741503	total: 2.59s	remaining: 399ms
26:	learn: 614093.6474392	total: 2.71s	remaining: 301ms
27:	learn: 609000.8148270	total: 2.78s	remaining: 199ms
28:	learn: 604738.4294844	total: 2.85s	remaining: 98.4ms
29:	learn: 601118.4172806	total: 2.91s	remaining: 0us
[CV]  od_wait=10, od_type=Iter, learning_rate=0.1, l2_leaf_reg=15, depth=8, score=-607493.829, total=   3.8s
[CV] od_wait=10, od_type=Iter, learning_rate=0.1, l2_leaf_reg=15, depth=8 
0:	learn: 1484946.9075269	total: 83.8ms	remaining: 2.43s
1:	learn: 1380019.6932316	total: 200ms	remaining: 2.8s
2:	learn: 1285211.0234251	total: 351ms	remaining: 3.16s
3:	learn: 1201138.0899862	total: 486ms	remaining: 3.16s
4:	learn: 1128025.6190806	total: 606ms	remaining: 3.03s
5:	learn: 1062142.3553781	total: 741

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 12.4min finished


0:	learn: 1358399.9138780	total: 317ms	remaining: 9.19s
1:	learn: 1171958.1039647	total: 970ms	remaining: 13.6s
2:	learn: 1021774.6341738	total: 1.2s	remaining: 10.8s
3:	learn: 911661.2236980	total: 1.52s	remaining: 9.91s
4:	learn: 827081.6567776	total: 1.86s	remaining: 9.3s
5:	learn: 764854.3137318	total: 2.43s	remaining: 9.73s
6:	learn: 717545.0706365	total: 2.77s	remaining: 9.09s
7:	learn: 680545.0371851	total: 3.08s	remaining: 8.47s
8:	learn: 652333.8179690	total: 3.42s	remaining: 7.99s
9:	learn: 630166.0999871	total: 3.72s	remaining: 7.45s
10:	learn: 612786.2211607	total: 4.43s	remaining: 7.66s
11:	learn: 598924.7877375	total: 4.79s	remaining: 7.18s
12:	learn: 588764.2124852	total: 5.06s	remaining: 6.61s
13:	learn: 580508.6527599	total: 5.46s	remaining: 6.24s
14:	learn: 573653.8037341	total: 6.01s	remaining: 6.01s
15:	learn: 567325.7519082	total: 6.37s	remaining: 5.57s
16:	learn: 561174.9480965	total: 6.7s	remaining: 5.12s
17:	learn: 556200.2801674	total: 6.98s	remaining: 4.66s
18

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<catboost.core.CatBoostRegressor object at 0x7fe2ee610450>,
                   iid='warn', n_iter=50, n_jobs=None,
                   param_distributions={'depth': [6, 8, 10],
                                        'l2_leaf_reg': [0, 3, 5, 10, 15, 50],
                                        'learning_rate': [0.05, 0.1, 0.2],
                                        'od_type': ['IncToDec', 'Iter'],
                                        'od_wait': [10, 30, 50, 100]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='neg_mean_absolute_error',
                   verbose=3)

In [24]:
display(rs_cv.best_params_)
from sklearn.externals import joblib
# Guarda el modelo
joblib.dump(rs_cv, 'catboost_rscv_1.pkl', compress=1)

{'od_wait': 50,
 'od_type': 'IncToDec',
 'learning_rate': 0.2,
 'l2_leaf_reg': 0,
 'depth': 10}

['catboost_rscv_1.pkl']

In [18]:
results = pd.DataFrame(rs_cv.cv_results_).sort_values('rank_test_score')

In [19]:
results[['param_learning_rate',
'param_depth',
'param_od_type',
'param_od_wait',
'param_l2_leaf_reg', 'mean_test_score', 'rank_test_score']].head(50)

,param_learning_rate,param_depth,param_od_type,param_od_wait,param_l2_leaf_reg,mean_test_score,rank_test_score
8,0.2,10,IncToDec,50,0,-539503.343352,1
0,0.2,10,Iter,30,5,-542606.004096,2
39,0.2,10,Iter,30,10,-543610.960921,3
14,0.2,10,IncToDec,10,50,-555048.163692,4
48,0.2,10,Iter,10,50,-555048.163692,4
6,0.2,10,IncToDec,100,50,-555048.163692,4
27,0.2,10,Iter,50,50,-555048.163692,4
23,0.2,8,Iter,100,0,-557099.682570,8
21,0.2,8,Iter,30,0,-557099.682570,8
4,0.2,8,Iter,10,10,-558429.067444,10


### hyperopt

In [ ]:

features =['habitaciones', 'banos','metroscubiertos', 'metrostotales', 'count_id_zona', 'antiguedad',
           'garages', 'gimnasio', 'usosmultiples', 'piscina', 'gimnasio',
           'promedio_id_zona', 'promedio_por_mes', 'promedio_precio_hbg_tipo_propiedad_provincia',
            'tam_ambientes', 'promedio_precio_tipo_propiedad_ciudad', 'distancia_centro_mexico',
            'distancia_ciudad_centrica', 'distancia_ciudad_cara', 'varianza_precio_ciudad', 
            'promedio_precio_habitaciones_banos_garages','promedio_precio_tipo_propiedad_ciudad']



x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.15)


D_train = catboost.Pool(x_train, y_train)
D_test = catboost.Pool(x_test, y_test)

def get_catboost_params(space):
    params = dict()
    params['learning_rate'] = space['learning_rate']
    params['depth'] = int(space['depth'])
    params['l2_leaf_reg'] = space['l2_leaf_reg']
    return params


obj_call_count = 0
cur_best_loss = np.inf
def objective(space):
    
    params = get_catboost_params(space)

    model = catboost.CatBoostClassifier(iterations=200,
                                        learning_rate=params['learning_rate'],
                                        depth=int(params['depth']),
                                        eval_metric='MAE',
                                        l2_leaf_reg=params['l2_leaf_reg'],
                                        early_stopping_rounds=3000,
                                        od_type="Iter",
                                        verbose= False
                                        )
    
    model.fit(D_train, eval_set=D_test, verbose= False)
    
    y_pred = model.predict_proba(D_test.get_features())
    
    return utils.MAE(df_test_f['precio'].values, y_pred)

In [ ]:
#  space = {
#          'depth': hp.quniform("depth", 1, 6, 10),
#          'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
#         'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
#         }


# best = hyperopt.fmin(fn=objective,
#                      space=space,
#                      algo=tpe.suggest,
#                      max_evals=200,
#                     verbose=1)

# print('-'*50)
# print('The best params:')
# print( best )

In [36]:
# features = ['habitaciones', 'garages','banos','antiguedad', 'metroscubiertos',  'metrostotales',  'provincia', 'tipodepropiedad',
#             'gimnasio', 'usosmultiples', 'piscina', 'fecha']

features = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'lat', 'lng', 'fecha', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'idf_titulo',
       'idf_descripcion', 'peso_titulo', 'peso_descripcion',
       'porcentaje_metros', 'diferencia_metros', 'metros_totales_normalizados',
       'metros_cubiertos_normalizados', 'escomercial',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
       'prop_frecuente', 'top_provincia', 'es_ciudad_centrica',
       'promedio_metros_totales_provincia',
       'promedio_metros_cubiertos_provincia', 'mes', 'dia', 'trimestre',
       'dias_desde_datos', 'meses_desde_datos',
       'cantidad_inquilinos', 'tam_ambientes', 'promedio_precio_provincia',
       'promedio_precio_ciudad', 'varianza_precio_ciudad', 'count_ciudad',
       'promedio_id_zona', 'promedio_id_zona_gen', 'varianza_id_zona',
       'count_id_zona', 'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'promedio_precio_tipo_propiedad_ciudad_gen', 'count_tipo_propiedad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes',
       'promedio_precio_habitaciones',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'lat_norm', 'lng_norm', 'promedio_precio_booleanos', 'puntaje',
       'distancia_ciudad_centrica', 'distancia_centro_mexico',
       'distancia_ciudad_cara'] + ['provincia', 'tipodepropiedad', 'ciudad']

# features_test = ['tam_ambientes','promedio_precio_tipo_propiedad_ciudad',
#                  'count_tipo_propiedad_ciudad','lat', 'lng',
#                     'varianza_precio_ciudad','puntaje',
#                  'distancia_ciudad_cara', 'distancia_ciudad_centrica' ,
#                  'distancia_ciudad_cara','promedio_id_zona',
#                  'promedio_por_mes', 'promedio_precio_hbg_tipo_propiedad_provincia',
#                  'promedio_precio_habitaciones_banos_garages',
#                 'ditancia_centro_bins_unif','ditancia_centro_bins_perc','distancia_centro_mexico_bins_unif',
#             'distancia_centro_mexico_bins_perc','distancia_ciudad_cara_bins_unif',
#             'distancia_ciudad_cara_bins_perc','promedio_id_zona_bins_unif', 'promedio_id_zona_bins_perc',
#             'promedio_precio_hbg_tipo_propiedad_provincia_bins_unif',
#             'promedio_precio_hbg_tipo_propiedad_provincia_bins_perc', 
#             'promedio_precio_tipo_propiedad_ciudad_bins_unif','promedio_precio_tipo_propiedad_ciudad_bins_perc',
#             'antiguedad_bins_unif', 'antiguedad_bins_perc', 'promedio_precio_hbg_tipo_propiedad_provincia_bins_perc',
#             'promedio_precio_hbg_tipo_propiedad_provincia_bins_unif', 'tam_ambientes_bins_perc',
#             'promedio_precio_hbg_tipo_propiedad_provincia_bins_unif']
    
# features += features_test

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.15)


hopts = {'od_wait': 50,
 'od_type': 'IncToDec',
 'learning_rate': 0.2,
 'l2_leaf_reg': 0,
 'depth': 10}


# train_labels = 'precio'
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=300,
                             eval_metric='MAE', **hopts)
# Fit model
model.fit(x_train, y_train, cat_features=['provincia', 'ciudad', 'tipodepropiedad'], eval_set=(x_test, y_test),plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1356588.2433262	test: 1354303.0974543	best: 1354303.0974543 (0)	total: 191ms	remaining: 57.2s
1:	learn: 1167684.3405885	test: 1167149.4387203	best: 1167149.4387203 (1)	total: 584ms	remaining: 1m 26s
2:	learn: 1020113.7522118	test: 1021487.4979983	best: 1021487.4979983 (2)	total: 1.09s	remaining: 1m 47s
3:	learn: 907462.0210939	test: 910216.5940042	best: 910216.5940042 (3)	total: 1.53s	remaining: 1m 53s
4:	learn: 824289.4214440	test: 829295.0907574	best: 829295.0907574 (4)	total: 1.78s	remaining: 1m 44s
5:	learn: 761222.1759372	test: 767830.2201183	best: 767830.2201183 (5)	total: 2.11s	remaining: 1m 43s
6:	learn: 711966.1133761	test: 719935.8914082	best: 719935.8914082 (6)	total: 2.5s	remaining: 1m 44s
7:	learn: 674651.6061637	test: 683863.1502465	best: 683863.1502465 (7)	total: 2.83s	remaining: 1m 43s
8:	learn: 647169.1448653	test: 657491.4249602	best: 657491.4249602 (8)	total: 3.08s	remaining: 1m 39s
9:	learn: 624316.5551783	test: 636012.5663081	best: 636012.5663081 (9)	tota

80:	learn: 443322.9959061	test: 505528.3701353	best: 505528.3701353 (80)	total: 16.3s	remaining: 44.1s
81:	learn: 442240.8815766	test: 505188.7879983	best: 505188.7879983 (81)	total: 16.5s	remaining: 43.8s
82:	learn: 441275.4824796	test: 505001.3249662	best: 505001.3249662 (82)	total: 16.6s	remaining: 43.4s
83:	learn: 440095.0837713	test: 503908.5326017	best: 503908.5326017 (83)	total: 17s	remaining: 43.6s
84:	learn: 439428.6243444	test: 503783.3101380	best: 503783.3101380 (84)	total: 17.3s	remaining: 43.7s
85:	learn: 438441.8524460	test: 503770.3671914	best: 503770.3671914 (85)	total: 17.4s	remaining: 43.4s
86:	learn: 437758.0415147	test: 503370.8344299	best: 503370.8344299 (86)	total: 17.6s	remaining: 43.1s
87:	learn: 436988.9483611	test: 503231.4287610	best: 503231.4287610 (87)	total: 17.7s	remaining: 42.7s
88:	learn: 435945.5695781	test: 503007.8188129	best: 503007.8188129 (88)	total: 17.9s	remaining: 42.5s
89:	learn: 434937.4264809	test: 502387.0698398	best: 502387.0698398 (89)	to

160:	learn: 382501.2149063	test: 493094.9500536	best: 493036.4457089 (159)	total: 31.2s	remaining: 27s
161:	learn: 381798.1606865	test: 493062.4724376	best: 493036.4457089 (159)	total: 31.6s	remaining: 26.9s
162:	learn: 381215.9252786	test: 493110.5278934	best: 493036.4457089 (159)	total: 31.9s	remaining: 26.8s
163:	learn: 380653.5604524	test: 492980.6703144	best: 492980.6703144 (163)	total: 32s	remaining: 26.5s
164:	learn: 380235.2470243	test: 492861.2973294	best: 492861.2973294 (164)	total: 32.2s	remaining: 26.3s
165:	learn: 379824.2260886	test: 492684.4731721	best: 492684.4731721 (165)	total: 32.3s	remaining: 26.1s
166:	learn: 379093.7610331	test: 492594.7387617	best: 492594.7387617 (166)	total: 32.5s	remaining: 25.9s
167:	learn: 378621.2813427	test: 492375.0143266	best: 492375.0143266 (167)	total: 32.6s	remaining: 25.6s
168:	learn: 377929.9888185	test: 492411.9415082	best: 492375.0143266 (167)	total: 32.8s	remaining: 25.4s
169:	learn: 377205.2868469	test: 492435.5816691	best: 49237

240:	learn: 343400.7009679	test: 489260.7644432	best: 488945.3859350 (232)	total: 46.9s	remaining: 11.5s
241:	learn: 342831.0405043	test: 489279.5966576	best: 488945.3859350 (232)	total: 47s	remaining: 11.3s
242:	learn: 342300.8983397	test: 489342.7730577	best: 488945.3859350 (232)	total: 47.2s	remaining: 11.1s
243:	learn: 341870.1306169	test: 489340.4503166	best: 488945.3859350 (232)	total: 47.3s	remaining: 10.9s
244:	learn: 341417.8878182	test: 489301.8213879	best: 488945.3859350 (232)	total: 47.5s	remaining: 10.7s
245:	learn: 340679.1607302	test: 489201.1435034	best: 488945.3859350 (232)	total: 47.6s	remaining: 10.5s
246:	learn: 340478.6321444	test: 489180.8090627	best: 488945.3859350 (232)	total: 47.8s	remaining: 10.3s
247:	learn: 340200.2962857	test: 489030.2203055	best: 488945.3859350 (232)	total: 48.2s	remaining: 10.1s
248:	learn: 339689.9607924	test: 488852.9675217	best: 488852.9675217 (248)	total: 48.3s	remaining: 9.9s
249:	learn: 339295.6867128	test: 488679.2185307	best: 4886

In [37]:
y_pred_test = model.predict(x_test)
y_pred_train = model.predict(x_train)
y_pred_eval = model.predict(utils.filtrar_features(df_test_f, features))

cat_mae_train = utils.MAE(y_train, y_pred_train)
cat_mae_test = utils.MAE(y_test, y_pred_test)
cat_mae_eval = utils.MAE(df_test_f['precio'], y_pred_eval)

print(f"MAE LightGBM (train): {cat_mae_train:.5f}")
print(f"MAE LightGBM (test): {cat_mae_test:.5f}")
print(f"MAE LightGBM (test): {cat_mae_eval:.5f}")

MAE LightGBM (train): 334780.71651
MAE LightGBM (test): 487956.82383
MAE LightGBM (test): 541690.14798
